Aim to complete parts of this tutorial on your own *before* the practical session.

Use the practical session to get help for any aspect you do not understand or were unable to complete.

# Dimensionality Reduction 2

Learning objectives
1. Apply MDS to different data sets and interpret the outputs using the popular python library [sklearn](https://scikit-learn.org/stable/)
2. Learn how to visualise the model outputs
3. Interpret the results and compare them with PCA

Optional learning objective (view the video online about NNMF before you start)

4. Apply NNMF to different data sets, visualise and interpret the output and compare them with PCA and MDS

## Import specific packages and functions

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import NMF, PCA
from sklearn.metrics import DistanceMetric
from sklearn.manifold import MDS

## Load in dataset
Read in the omics datasets using the pandas [read_excel()](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) function. For this example we will be using a urine metabolomics data from a diabetes cohort. 

In [ ]:
diabetes_urine_metab = pd.read_excel("../Data-main/diabetes_metabolomics_urine.xlsx")

In [ ]:
diabetes_urine_metab

We can see there are several covariates in the first columns: Age, Gender, BMI, ETH, and T2D (diabetes status). To obtain just the metabolite relative abundance data, we can use `diabetes_urine_metab.iloc[:, 6:]`. This takes all rows of the data, and all columns after the 6th column.

## Multidimensional Scaling (MDS)

[Multidimensional scaling](https://scikit-learn.org/0.24/modules/manifold.html#multidimensional-scaling) (MDS) seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space. MDS (both metric and non-metric) can be performed using the [sklearn.MDS()](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html) function.

Some algorithms make use of a random process, e.g. to initialise weights it select initial starting points from a random number generator. This means that each time you use the algorithm, the answer might be slightly (sometimes this is unnoticeable) different. Also, this may mean the output you get is different from your neighbour.

How do you get the same answer each time (as long as you do not change any other parameters)? You can set the random state to a specific number.

Random number generators use the computer time and other factors to start at a random place in a huge sequence of random numbers. By setting the ```random_state``` property of an algorithm you will ensure it will start at a standard place. You can pick any positive integer for this. If you look at the documentation of [PCA](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html?highlight=pca#sklearn.decomposition.PCA) and [MDS](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.MDS.html) you will see both of these methods have this argument as input, with a default value, however using other default parameters PCA will give the same output whereas the output from MDS might change.

In [ ]:
# enter your CID here, or date of birth, or another number of your choosing to use as random state
CID = 0 # do not add the leading 0

# remember to check the documentation of each algorithm if setting the random_state is needed
# for this tutorial and all upcoming tutorials...

### Metric MDS
Set the `metric` parameter to True to perform metric MDS

In [ ]:
embedding = MDS(n_components=2, metric=True, random_state=CID)

Apply the `fit_transform` function to the MDS object to transform the data. By default metric MDS uses Euclidean distances in the sklearn implementation we are using:

In [ ]:
embedding_euclidean = embedding.fit_transform(diabetes_urine_metab.iloc[:, 6:])

Visualise the first two MDS components:

In [ ]:
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=embedding_euclidean[:, 0], y=embedding_euclidean[:, 1], hue=diabetes_urine_metab["T2D"])

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")
plt.title("Metric MDS")
plt.show()

### Dissimilarity metrics
We can use different metrics to compute the dissimilarity matrix for metric MDS. The default option used in the MDS function is Euclidean distance. We can otherwise specify `dissimilarity = "precomputed"`. Here we provide a dissimilarity matrix we have already computed. 

We can compute the dissimilarity matrix using [various metrics](https://scikit-learn.org/0.24/modules/generated/sklearn.neighbors.DistanceMetric.html).

#### Running MDS on a precomputed Manhattan distance dissimilarity matrix

In [ ]:
# compute manhattan distances on the data
manhattan_dist = DistanceMetric.get_metric('manhattan').pairwise(diabetes_urine_metab.iloc[:, 6:])

# perform metric MDS with the manhattan distances
embedding_manhattan = MDS(n_components=2, metric=True, dissimilarity="precomputed")
MDS_manhattan = embedding_manhattan.fit_transform(manhattan_dist)

#### and Minkowski distance

In [ ]:
minkowski_dist = DistanceMetric.get_metric('minkowski').pairwise(diabetes_urine_metab.iloc[:, 6:])

embedding_minkowski = MDS(n_components=2, metric=True, dissimilarity="precomputed")
MDS_minkowski = embedding_minkowski.fit_transform(minkowski_dist)

#### Compare the results obtained using metric MDS with different dissimilarity metrics:

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=embedding_euclidean[:, 0], y=embedding_euclidean[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax1)
ax1.set_title("Euclidean")

sns.scatterplot(x=MDS_manhattan[:, 0], y=MDS_manhattan[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax2)
ax2.set_title("Manhattan")

sns.scatterplot(x=MDS_minkowski[:, 0], y=MDS_minkowski[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax3)
ax3.set_title("Minkowski")

plt.tight_layout()
plt.show()

Using previous code examples, compute another similarity metric based on the metabolomics data (full list [here](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.DistanceMetric.html)) and visualise the first two components using a scatterplot:

In [ ]:
# Write your code here...

### Non-metric MDS
To compute non-metric MDS, set the parameter `metric=False`

In [ ]:
embedding_nonmetric = MDS(n_components=2, metric=False)
diabetes_urine_metab_transformed_nMDS = embedding_nonmetric.fit_transform(diabetes_urine_metab.iloc[:, 6:])

Visualise the first two non-metric MDS components:

In [ ]:
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=diabetes_urine_metab_transformed_nMDS[:, 0],
 y=diabetes_urine_metab_transformed_nMDS[:, 1], 
 hue=diabetes_urine_metab["T2D"])

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")
plt.title("Non-metric MDS")
plt.show()

## How does MDS compare with PCA?

Using what you have learnt from the previous tutorial on PCA, scale the input metabolomics data, apply PCA and project the metabolomics data into the latent space, and visualise the data using a scatterplot. Colour the datapoints by Type 2 diabetes status "T2D".

In [ ]:
# import scaler
from sklearn.preprocessing import StandardScaler

# scale the data
diabetes_urine_metab_scaled = StandardScaler().fit_transform(diabetes_urine_metab.iloc[:, 6:])

In [ ]:
# perform PCA and project the data
pca_res = PCA(n_components=2).fit_transform(diabetes_urine_metab_scaled)

In [ ]:
# plot two components of the projected data on a scatter plot
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=pca_res[:, 0],
 y=pca_res[:, 1],
 hue=diabetes_urine_metab["T2D"])

p.set_xlabel("PC1")
p.set_ylabel("PC2")

plt.title("PCA")

plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

sns.scatterplot(x=pca_res[:, 0], y=pca_res[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax1)
ax1.set_title("PCA")

sns.scatterplot(x=embedding_euclidean[:, 0], y=embedding_euclidean[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax2)
ax2.set_title("MDS (metric)")

plt.tight_layout()
plt.show()

How similar are these projections?

## Your turn
Choose one of the other omics datasets in the `Data` folder and perform MDS on it. Visualise the results.

In [ ]:
# Import dataset

In [ ]:
# Perform MDS

In [ ]:
# Visualise the results

# Optional materials
Please view the video online about NNMF before you start this part of the tutorial.

## Non-negative Matrix Factorisation (NMF)

We can use the [sklearn.NMF() function](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.NMF.html) to perform non-negative matrix factorisation on the dataset. We will initialise the model with five components, and use the "random" initialiser.

The input matrix $X$ (the metabolomics dataset without covariates/metadata) will be factorised into the matrices $W$ (weights / projected data) and $H$ (hidden factorization / principal components)

$X \approx HW$

Note: the original publication uses a different formula and it assumes that the data (V instead of X) was transposed (each row are variables and each column is a sample) compared to here.

In [ ]:
# instantiate the NMF model
# here we will select 5 components
model = NMF(n_components=5, init='random', random_state=CID)

# set X as the input omics data matrix
X = diabetes_urine_metab.iloc[:, 6:]

# the hidden components matrix H is obtained by projecting the original data
H = model.fit_transform(X)

# the weights matrix W is obtained by using the .components_ attribute
W = model.components_


(If you get a warning, take a look in the documentation on what this is and how to add it as an input to the function)

Let's look a the shape of the resulting matrices:

In [ ]:
H.shape

In [ ]:
W.shape

We should be able to approximately reconstruct the original matrix $V$ by calculating the product of $W$ and $H$

In [ ]:
# obtain the dot product of the matrices W and H
X_approx = pd.DataFrame(np.dot(H, W))

# This is the same shape as the original matrix V
X_approx.shape

We can compare the two matrices V and V_approx by eye:

In [ ]:
X_approx

In [ ]:
X

We can also estimate the reconstruction error using the Frobenius norm:

In [ ]:
model.reconstruction_err_

How does the reconstruction error change as you vary `n_components`?

### Interpreting NMF

The $W$ matrix contains the loadings or principal components, summarising the importance of each feature (metabolites in this case) in the new components, of which we have selected 5.

In [ ]:
# add metabolite names to the matrix W to help interpretation
W_df = pd.DataFrame(W.T, columns=range(1, W.shape[0]+1), index=X.columns)

In [ ]:
W_df

If we sort each column, we can determine the most important metabolites driving each component (1-5)


In [ ]:
# for each component, print out the top 5 features driving that component
for col in W_df:
    print("Component: "+ str(col))
    print(W_df[col].sort_values(ascending=False)[0:5].index.tolist())

Visualise the first (not top) 20 metabolites in the metabolomics data, alongside their contribution to the 5 latent factors:

In [ ]:
plt.figure(figsize=(8, 8))
plt.imshow(W_df.iloc[0:20, :], cmap="viridis")
plt.yticks(range(len(W_df.iloc[0:20, :].index)), W_df.iloc[0:20, :].index)
plt.xticks(range(0, 5), [1, 2, 3, 4, 5])
plt.colorbar()
plt.show()

Visualise the first two components of the NMF projected data ($H$) using a scatterplot:

In [ ]:
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=H[:, 0],
 y=H[:, 1], 
 hue=diabetes_urine_metab["T2D"])

p.set_xlabel("Dim 1")
p.set_ylabel("Dim 2")

plt.show()

We can also plot a heatmap using the NMF projected data $H$, which clusters the samples based on T2D status using the five latent factors:

In [ ]:
sns.clustermap(H.T,
col_colors=["tab:orange" if i == 1 else "tab:blue" for i in diabetes_urine_metab["T2D"]],
row_colors=sns.color_palette("tab10"),
xticklabels=False)

plt.show()

## How do NMF and MDS compare with PCA?

Using what you have learnt from the previous tutorial on PCA, scale the input metabolomics data, apply PCA and project the metabolomics data into the latent space, and visualise the data using a scatterplot. Colour the datapoints by Type 2 diabetes status "T2D".

In [ ]:
# import scaler
from sklearn.preprocessing import StandardScaler

# scale the data
diabetes_urine_metab_scaled = StandardScaler().fit_transform(diabetes_urine_metab.iloc[:, 6:])

In [ ]:
# perform PCA and project the data
pca_res = PCA(n_components=2).fit_transform(diabetes_urine_metab_scaled)

In [ ]:
# plot two components of the projected data on a scatter plot
sns.set_style("ticks")
sns.set_context("notebook")
plt.figure(figsize=(8, 8))

p = sns.scatterplot(x=pca_res[:, 0],
 y=pca_res[:, 1],
 hue=diabetes_urine_metab["T2D"])

p.set_xlabel("PC1")
p.set_ylabel("PC2")

plt.title("PCA")

plt.show()

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))

sns.scatterplot(x=pca_res[:, 0], y=pca_res[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax1)
ax1.set_title("PCA")

sns.scatterplot(x=H[:, 0], y=H[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax2)
ax2.set_title("NMF")

sns.scatterplot(x=embedding_euclidean[:, 0], y=embedding_euclidean[:, 1], hue=diabetes_urine_metab["T2D"], ax=ax3)
ax3.set_title("MDS (metric)")

plt.tight_layout()
plt.show()

Note - look at the axes: using NMF all weights are POSITIVE, unlike in PCA or MDS